# Machine Learning–Based Prediction of Structural MRI Measures from Military Environmental Exposures in Veterans

## 1. Data cleaning

In [4]:
import numpy as np
import pandas as pd


### 1a. Load Data

In [5]:
#Load original data
def load_data(filepath):
    path = filepath
    df = pd.read_csv(path)
    return df
     
#load_data(r'WRIISC_data\me_data.csv')
#load_data(r'WRIISC_data\mri_data.csv')

In [6]:
#Create copy of original data
def create_copy(path):
    data = load_data(path)
    df = data.copy()
    return df

In [ ]:
# Create a copy of Intake Packet
df_intake = create_copy(r'WRIISC_data\intake_packet.csv') 

#### Extract exposure (binary) columns

In [ ]:
# Extract military exposure ('expo_') columns AND 'WCA' column
# First check the columns
me_columns = [col for col in df_intake.columns if col.startswith('expo_') or col == 'WCA']

In [ ]:
# Extract columns only before 'expo_meta_c'
if 'expo_meta_c' in df_intake.columns:
    idx = df_intake.columns.get_loc('expo_meta_c')
    me_columns = df_intake.columns[:idx + 1].tolist()  
else:
    print("'expo_meta_c' not found in DataFrame columns.")


In [12]:
# Check the extracted columns
print(me_columns)

['ID', 'WCA', 'dem_dob', 'dem_sex', 'dem_datecom', 'dem_race01', 'dem_race02', 'dem_race03', 'dem_race04', 'dem_race05', 'dem_race06', 'dem_race07', 'dem_raceoth', 'dem_ethnic', 'dem_educ_str', 'dem_educ', 'dem_educ01', 'dem_educ02', 'dem_educ03', 'dem_educ04', 'dem_educ05', 'dem_educ06', 'dem_educ07', 'dem_educ08', 'dem_educ09', 'dem_educ_oth', 'dem_lang1', 'dem_lang2', 'dem_marital', 'dem_mar_length', 'dem_mar_times', 'dem_mar_health', 'dem_emp01', 'dem_emp02', 'dem_emp03', 'dem_emp04', 'dem_emp05', 'dem_emp06', 'dem_emp07', 'dem_emp08', 'dem_emp_oth', 'dem_handed', 'mil_branch01_start', 'mil_branch01_end', 'mil_branch02_start', 'mil_branch02_end', 'mil_branch03_start', 'mil_branch03_end', 'mil_branch04_start', 'mil_branch04_end', 'mil_branch05_start', 'mil_branch05_end', 'mil_branch06_start', 'mil_branch06_end', 'mil_branch07_start', 'mil_branch07_end', 'mil_branch08_start', 'mil_branch08_end', 'mil_branch09_start', 'mil_branch09_end', 'mil_branch10_start', 'mil_branch10_end', 'mil_

In [13]:
# Check the extracted columns
df_me = df_intake[me_columns]
me_columns = [col for col in df_me.columns if 
              col.startswith('expo_') or 
              col == 'WCA' or 
              col == 'dem_datecom']
df_me = df_intake[me_columns]

# Sort df_me by 'WCA' column
df_me = df_me.sort_values(by='WCA')

# Change 'WCA' column to 'SubjectID'
df_me.rename(columns={'WCA': 'SubjectID'}, inplace=True)

In [14]:
# Drop all columns ending with '_f', '_i' and '_c'
cols_to_drop = [col for col in df_me.columns if col.endswith(('_f', '_i', '_c'))]
df_me.drop(columns=cols_to_drop, inplace=True)

#### Extract covariates

In [15]:
# Extract age, gender and tbi

# Check the columns
print(df_intake.columns)
print(df_intake.columns[df_intake.columns.str.contains('tbi_')])  

# Extract covariates
covariates = [col for col in df_intake.columns 
              if col == 'dem_dob' or
              col == 'dem_sex' or
              col.startswith('tbi_') or
              col == 'WCA' or
              col == 'dem_datecom']

# Extract covariates from df
df_covariates = df_intake[covariates]

# Sort df_me by 'WCA' column
df_covariates = df_covariates.sort_values(by='WCA')

# Change 'WCA' column to 'SubjectID' and 'dem_sex' to 'Sex'
df_covariates.rename(columns={'WCA': 'SubjectID'}, inplace=True)
df_covariates.rename(columns={'dem_sex' : 'Sex'}, inplace=True)

Index(['ID', 'WCA', 'dem_dob', 'dem_sex', 'dem_datecom', 'dem_race01',
       'dem_race02', 'dem_race03', 'dem_race04', 'dem_race05',
       ...
       'cds_24', 'cds_26', 'cds_28', 'cds_29', 'cds_30', 'cds_33', 'cds_35',
       'cds_36', 'cds_37', 'Site'],
      dtype='object', length=871)
Index(['tbi_01a', 'tbi_01b', 'tbi_01c', 'tbi_01d', 'tbi_01e', 'tbi_01f',
       'tbi_02a', 'tbi_02b', 'tbi_02c', 'tbi_02d', 'tbi_02e', 'tbi_03a',
       'tbi_03b', 'tbi_03c', 'tbi_03d', 'tbi_03e', 'tbi_03f', 'tbi_04a',
       'tbi_04b', 'tbi_04c', 'tbi_04d', 'tbi_04e', 'tbi_04f', 'tbi_05',
       'tbi_05_spec'],
      dtype='object')


Drop columns with text, labeled  as 999. 

In [16]:
df_covariates.shape

(420, 29)

In [17]:
df_covariates.columns


Index(['SubjectID', 'dem_dob', 'Sex', 'dem_datecom', 'tbi_01a', 'tbi_01b',
       'tbi_01c', 'tbi_01d', 'tbi_01e', 'tbi_01f', 'tbi_02a', 'tbi_02b',
       'tbi_02c', 'tbi_02d', 'tbi_02e', 'tbi_03a', 'tbi_03b', 'tbi_03c',
       'tbi_03d', 'tbi_03e', 'tbi_03f', 'tbi_04a', 'tbi_04b', 'tbi_04c',
       'tbi_04d', 'tbi_04e', 'tbi_04f', 'tbi_05', 'tbi_05_spec'],
      dtype='object')

In [18]:
# Drop columns
cols_to_drop = [col for col in df_covariates.columns if col == 'tbi_01f' or col == 'tbi_05_spec']
df_covariates.drop(columns=cols_to_drop, inplace=True)

#### Extract MRI

In [19]:
# Create copy of original data and preview the first few rows of the MRI data
df_mri = create_copy(r'WRIISC_data\mri_data.csv')
df_mri.shape

(426, 203)

In [20]:
# Print the columns that have majority 0
cols_with_majority_0 = [col for col in df_mri.columns if (df_mri[col] == 0).mean() > 0.5]
print("Columns with majority 0:")
print(cols_with_majority_0) 

Columns with majority 0:
['5th-Ventricle', 'Left-WM-hypointensities', 'Right-WM-hypointensities', 'non-WM-hypointensities', 'Left-non-WM-hypointensities', 'Right-non-WM-hypointensities']


In [21]:
# Drop columns with majority 0
df_mri.drop(columns=cols_with_majority_0, inplace=True)

#### Extract sub-exposure (binary) columns

In [22]:
# Extract military exposure ('expo_') columns AND 'WCA' column

# First check the columns
sub_me_columns = [col for col in df_intake.columns if col.startswith('expo_') or col == 'WCA']
print(sub_me_columns)

['WCA', 'expo_herb', 'expo_herb_f', 'expo_herb_i', 'expo_herb_c', 'expo_bite', 'expo_bite_f', 'expo_bite_i', 'expo_bite_c', 'expo_anim', 'expo_anim_f', 'expo_anim_i', 'expo_anim_c', 'expo_anth', 'expo_anth_f', 'expo_anth_i', 'expo_anth_c', 'expo_asbe', 'expo_asbe_f', 'expo_asbe_i', 'expo_asbe_c', 'expo_biow', 'expo_biow_f', 'expo_biow_i', 'expo_biow_c', 'expo_mopp', 'expo_mopp_f', 'expo_mopp_i', 'expo_mopp_c', 'expo_napp', 'expo_napp_f', 'expo_napp_i', 'expo_napp_c', 'expo_wche', 'expo_wche_f', 'expo_wche_i', 'expo_wche_c', 'expo_chem', 'expo_chem_f', 'expo_chem_i', 'expo_chem_c', 'expo_chlo', 'expo_chlo_f', 'expo_chlo_i', 'expo_chlo_c', 'expo_npw', 'expo_npw_f', 'expo_npw_i', 'expo_npw_c', 'expo_wash', 'expo_wash_f', 'expo_wash_i', 'expo_wash_c', 'expo_vibr', 'expo_vibr_f', 'expo_vibr_i', 'expo_vibr_c', 'expo_fogo', 'expo_fogo_f', 'expo_fogo_i', 'expo_fogo_c', 'expo_bloo', 'expo_bloo_f', 'expo_bloo_i', 'expo_bloo_c', 'expo_poll', 'expo_poll_f', 'expo_poll_i', 'expo_poll_c', 'expo_infe

In [23]:
# Extract columns only before 'expo_meta_c'
if 'expo_meta_c' in df_intake.columns:
    idx = df_intake.columns.get_loc('expo_meta_c')
    sub_me_columns = df_intake.columns[:idx + 1].tolist()  
else:
    print("'expo_meta_c' not found in DataFrame columns.")


In [24]:
# Check the extracted columns
df_sub_me = df_intake[sub_me_columns]
sub_me_columns = [col for col in df_sub_me.columns if 
              col.startswith('expo_') or 
              col == 'WCA' or 
              col == 'dem_datecom']
df_sub_me = df_intake[sub_me_columns]

# Sort df_me by 'WCA' column
df_sub_me = df_sub_me.sort_values(by='WCA')

# Change 'WCA' column to 'SubjectID'
df_sub_me.rename(columns={'WCA': 'SubjectID'}, inplace=True)

### 1b. Remove Duplicates

Note: Some patients have multiple WCA subject Ids indicating different times at which intake packet was completed. Choose most recent timestamp.

In [25]:
# Create function to remove duplicates    
def remove_duplicates(df):
    # Standardize SubjectID
    df['SubjectID'] = (
        df['SubjectID']
        .astype(str)
        .str.upper()
        .replace(r'(_T1|_T2|_T3)$', '', regex=True)
        .str.strip()
    )

    # If more than one SubjectID, keep the most recent dem_datecom
    if 'dem_datecom' in df.columns:
        df['dem_datecom'] = pd.to_datetime(df['dem_datecom'], errors='coerce')
        df = df.sort_values(['SubjectID', 'dem_datecom'], ascending=[True, False])
        df_2 = df.drop_duplicates(subset='SubjectID', keep='first')
    else:
        df_2 = df.drop_duplicates(subset='SubjectID', keep='first')

    # Print shape before/after
    print(f"DataFrame shape before removing duplicates: {df.shape}")
    print(f"DataFrame shape after removing duplicates: {df_2.shape}")

    return df_2  # return the de-duplicated DataFrame


In [26]:
# Remove duplicates from the military exposure data
# Note: This will keep the most recent 'dem_datecom' for each 'SubjectID
df_me = remove_duplicates(df_me)

# Remove dem_datecom column
df_me = df_me.drop(columns=['dem_datecom'], errors='ignore')


DataFrame shape before removing duplicates: (420, 37)
DataFrame shape after removing duplicates: (407, 37)


In [27]:
# Remove duplicates from the covariates data
df_covariates = remove_duplicates(df_covariates)

# Remove dem_datecom column
df_covariates = df_covariates.drop(columns=['dem_datecom'], errors='ignore')


DataFrame shape before removing duplicates: (420, 27)
DataFrame shape after removing duplicates: (407, 27)


In [28]:
# Remove duplicates from MRI data
df_mri = remove_duplicates(df_mri)



DataFrame shape before removing duplicates: (426, 197)
DataFrame shape after removing duplicates: (426, 197)


In [29]:
# Remove duplicates from the sub-military exposure data
# Note: This will keep the most recent 'dem_datecom' for each 'SubjectID
df_sub_me = remove_duplicates(df_sub_me)

# Remove dem_datecom column
df_sub_me = df_sub_me.drop(columns=['dem_datecom'], errors='ignore')


DataFrame shape before removing duplicates: (420, 142)
DataFrame shape after removing duplicates: (407, 142)


### 1c. Convert missing values to NaN.

#### Calculate number of missing values.

Covert exposure to binary. 

In [30]:
# Convert exposure columns to binary in ME dataset
target_cols = [col for col in df_me.columns if col.startswith('expo_')]
df_me[target_cols] = df_me[target_cols].replace({0: 0, 1: 1, 2: 1})


In [31]:
# Calculate number of missing values in data sets.
def calculate_missing_values(df):
    missing_values = df.isnull().sum()
    total_cells = np.prod(df.shape)
    total_missing = missing_values.sum()
    percent_missing = (total_missing / total_cells) * 100
    print(f"Total missing values: {total_missing}")
    print(f"Percentage of missing values: {percent_missing:.2f}%")
    return missing_values

In [32]:
# Convert missing values and 999.0 to NaN 
def convert_missing_values(df):
    df = df.replace(["", " ", 999.0], np.nan)
    return df

##### For df_me.

In [33]:
calculate_missing_values(df_me)

Total missing values: 53
Percentage of missing values: 0.36%


SubjectID    0
expo_herb    1
expo_bite    1
expo_anim    2
expo_anth    1
expo_asbe    5
expo_biow    3
expo_mopp    3
expo_napp    1
expo_wche    1
expo_chem    1
expo_chlo    3
expo_npw     2
expo_wash    2
expo_vibr    1
expo_fogo    1
expo_bloo    2
expo_poll    2
expo_infe    1
expo_inse    1
expo_pest    1
expo_radi    1
expo_lase    1
expo_nois    2
expo_pain    1
expo_petr    1
expo_wexh    1
expo_prop    2
expo_rada    1
expo_sand    2
expo_burn    1
expo_oilf    1
expo_heat    1
expo_vacc    1
expo_du      1
expo_meta    1
dtype: int64

In [34]:
# Convert missing values and 999.0 to NaN in exposure data set
df_me = convert_missing_values(df_me)

In [35]:
calculate_missing_values(df_me)

Total missing values: 2887
Percentage of missing values: 19.70%


SubjectID      0
expo_herb     26
expo_bite    101
expo_anim    140
expo_anth     74
expo_asbe     75
expo_biow     72
expo_mopp    136
expo_napp    110
expo_wche     31
expo_chem     40
expo_chlo    132
expo_npw      61
expo_wash     66
expo_vibr    146
expo_fogo    149
expo_bloo    142
expo_poll    146
expo_infe     68
expo_inse     30
expo_pest     42
expo_radi     61
expo_lase     61
expo_nois     73
expo_pain     73
expo_petr     68
expo_wexh     74
expo_prop     32
expo_rada     70
expo_sand     83
expo_burn     91
expo_oilf     66
expo_heat    144
expo_vacc     34
expo_du       37
expo_meta    133
dtype: int64

##### For df_covariates.

Change tbi to binary.

In [36]:
# Convert tbi columns to binary (i.e. False = 0, True = 1)
tbi_columns = [col for col in df_covariates.columns if col.startswith('tbi_')]
recode_map = {False: 0, True: 1}
for col in tbi_columns:
    if col in df_covariates.columns:
        df_covariates[col] = df_covariates[col].map(recode_map)
    # If enire column is NaN, drop the column
    if df_covariates[col].isnull().all():
        df_covariates = df_covariates.drop(columns=[col], errors='ignore')



Convert dem_dob to Age

In [37]:
from datetime import datetime

if 'dem_dob' in df_covariates.columns:
    # Convert to datetime
    df_covariates['dem_dob'] = pd.to_datetime(
    df_covariates['dem_dob'].astype(str).str.strip(),
    format='%d%b%Y %H:%M:%S',
    errors='coerce'
)
    # Set 1/1/1900 as missing
    df_covariates.loc[df_covariates['dem_dob'] == pd.Timestamp('1900-01-01'), 'dem_dob'] = pd.NaT
    # Calculate age
    current_year = datetime.now().year
    df_covariates['Age'] = current_year - df_covariates['dem_dob'].dt.year
    # If dem_dob is missing, set Age as missing
    df_covariates.loc[df_covariates['dem_dob'].isna(), 'Age'] = np.nan
    # Drop dem_dob column
    df_covariates= df_covariates.drop(columns=['dem_dob'], errors='ignore')
    # Move 'Age' to third column
    cols = df_covariates.columns.tolist()
    age_col = cols.pop(cols.index('Age'))
    cols.insert(2, age_col)
    df_covariates = df_covariates[cols]


In [38]:
calculate_missing_values(df_covariates)

Total missing values: 610
Percentage of missing values: 5.76%


SubjectID     0
Sex           0
Age           1
tbi_01a       3
tbi_01b       2
tbi_01c       2
tbi_01d       2
tbi_01e       2
tbi_02a      18
tbi_02b      19
tbi_02c      18
tbi_02d      18
tbi_02e      18
tbi_03a      35
tbi_03b      35
tbi_03c      35
tbi_03d      35
tbi_03e      35
tbi_03f      35
tbi_04a      49
tbi_04b      49
tbi_04c      49
tbi_04d      49
tbi_04e      49
tbi_04f      49
tbi_05        3
dtype: int64

In [39]:
df_covariates = convert_missing_values(df_covariates)

In [40]:
calculate_missing_values(df_covariates) 

Total missing values: 610
Percentage of missing values: 5.76%


SubjectID     0
Sex           0
Age           1
tbi_01a       3
tbi_01b       2
tbi_01c       2
tbi_01d       2
tbi_01e       2
tbi_02a      18
tbi_02b      19
tbi_02c      18
tbi_02d      18
tbi_02e      18
tbi_03a      35
tbi_03b      35
tbi_03c      35
tbi_03d      35
tbi_03e      35
tbi_03f      35
tbi_04a      49
tbi_04b      49
tbi_04c      49
tbi_04d      49
tbi_04e      49
tbi_04f      49
tbi_05        3
dtype: int64

##### Check for df_mri

In [41]:
calculate_missing_values(df_mri)

Total missing values: 0
Percentage of missing values: 0.00%


SubjectID                         0
Left-Lateral-Ventricle            0
Left-Inf-Lat-Vent                 0
Left-Cerebellum-White-Matter      0
Left-Cerebellum-Cortex            0
                                 ..
ThickAvg_rh_supramarginal         0
ThickAvg_rh_frontalpole           0
ThickAvg_rh_temporalpole          0
ThickAvg_rh_transversetemporal    0
ThickAvg_rh_insula                0
Length: 197, dtype: int64

#### For df_sub_me

In [42]:
calculate_missing_values(df_sub_me)

Total missing values: 5619
Percentage of missing values: 9.79%


SubjectID       0
expo_herb       1
expo_herb_f    77
expo_herb_i    77
expo_herb_c    77
               ..
expo_du_c      69
expo_meta       1
expo_meta_f    84
expo_meta_i    84
expo_meta_c    83
Length: 141, dtype: int64

In [43]:
# Convert missing values and 999.0 to NaN in sub_exposure data set
df_sub_me = convert_missing_values(df_sub_me)

In [44]:
calculate_missing_values(df_sub_me)

Total missing values: 28971
Percentage of missing values: 50.48%


SubjectID        0
expo_herb       26
expo_herb_f    346
expo_herb_i    350
expo_herb_c    313
              ... 
expo_du_c      256
expo_meta      133
expo_meta_f    380
expo_meta_i    379
expo_meta_c    377
Length: 141, dtype: int64

## 2. Data preprocessing

In [45]:
# Merge df_sub_me with df_mri and df_covariates
df_final_sub_me = pd.merge(df_sub_me, df_mri, on='SubjectID', how='outer')
df_final_sub_me = pd.merge(df_final_sub_me, df_covariates, on='SubjectID', how='outer')   

##### Merge Data into final dataset

In [46]:
# Check shape of each cleaned dataset
print(f"Shape of cleaned ME data: {df_me.shape}")   
print(f"Shape of cleaned Covariates data: {df_covariates.shape}")
print(f"Shape of cleaned MRI data: {df_mri.shape}")

Shape of cleaned ME data: (407, 36)
Shape of cleaned Covariates data: (407, 26)
Shape of cleaned MRI data: (426, 197)


In [47]:
# Save the dataframes to CSV files
df_me.to_csv(r'WRIISC_data\cleaned_me_data.csv', index=False)
df_covariates.to_csv(r'WRIISC_data\cleaned_covariates_data.csv', index=False)
df_mri.to_csv(r'WRIISC_data\cleaned_mri_data.csv', index=False)

In [48]:
def merge_data(cleaned_me_path, cleaned_mri_path, cleaned_covariates_path): 
    df_me = pd.read_csv(cleaned_me_path)
    df_mri = pd.read_csv(cleaned_mri_path)
    df_covariates = pd.read_csv(cleaned_covariates_path)

    # Merge ME and MRI first, then merge with covariates
    df = pd.merge(df_me, df_mri, on='SubjectID', how='inner')
    df = pd.merge(df, df_covariates, on='SubjectID', how='inner')

    return df

In [49]:
# Load the data
df = merge_data(r'WRIISC_data\cleaned_me_data.csv', 
                       r'WRIISC_data\cleaned_mri_data.csv', 
                       r'WRIISC_data\cleaned_covariates_data.csv')



In [50]:
df.columns.to_list()

['SubjectID',
 'expo_herb',
 'expo_bite',
 'expo_anim',
 'expo_anth',
 'expo_asbe',
 'expo_biow',
 'expo_mopp',
 'expo_napp',
 'expo_wche',
 'expo_chem',
 'expo_chlo',
 'expo_npw',
 'expo_wash',
 'expo_vibr',
 'expo_fogo',
 'expo_bloo',
 'expo_poll',
 'expo_infe',
 'expo_inse',
 'expo_pest',
 'expo_radi',
 'expo_lase',
 'expo_nois',
 'expo_pain',
 'expo_petr',
 'expo_wexh',
 'expo_prop',
 'expo_rada',
 'expo_sand',
 'expo_burn',
 'expo_oilf',
 'expo_heat',
 'expo_vacc',
 'expo_du',
 'expo_meta',
 'Left-Lateral-Ventricle',
 'Left-Inf-Lat-Vent',
 'Left-Cerebellum-White-Matter',
 'Left-Cerebellum-Cortex',
 'Left-Thalamus-Proper',
 'Left-Caudate',
 'Left-Putamen',
 'Left-Pallidum',
 '3rd-Ventricle',
 '4th-Ventricle',
 'Brain-Stem',
 'Left-Hippocampus',
 'Left-Amygdala',
 'CSF',
 'Left-Accumbens-area',
 'Left-VentralDC',
 'Left-vessel',
 'Left-choroid-plexus',
 'Right-Lateral-Ventricle',
 'Right-Inf-Lat-Vent',
 'Right-Cerebellum-White-Matter',
 'Right-Cerebellum-Cortex',
 'Right-Thalamus-Pr

In [51]:
df.shape

(263, 257)

## 3. Machine Learning

In [52]:
# Targets: "exclude expo_*, age, sex, and tbi_* columns
target = [
    col for col in df.columns
    if not col.startswith('expo_') and col not in ['age', 'sex'] and not col.startswith('tbi_')
]
[col for col in df.columns if col.startswith('expo_')]

# Features: columns starting with "expo_"
feature = [col for col in df.columns if col.startswith('expo_')]

print(target)
print(feature)

# Extract covariates 
covariate_cols = [col for col in df.columns if col in df_covariates.columns and col not in target_cols and col != 'SubjectID']
print(covariate_cols)


['SubjectID', 'Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent', 'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex', 'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum', '3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'Left-Hippocampus', 'Left-Amygdala', 'CSF', 'Left-Accumbens-area', 'Left-VentralDC', 'Left-vessel', 'Left-choroid-plexus', 'Right-Lateral-Ventricle', 'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper', 'Right-Caudate', 'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus', 'Right-Amygdala', 'Right-Accumbens-area', 'Right-VentralDC', 'Right-vessel', 'Right-choroid-plexus', 'WM-hypointensities', 'Optic-Chiasm', 'CC_Posterior', 'CC_Mid_Posterior', 'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior', 'BrainSegVol', 'BrainSegVolNotVent', 'BrainSegVolNotVentSurf', 'lhCortexVol', 'rhCortexVol', 'CortexVol', 'lhCerebralWhiteMatterVol', 'rhCerebralWhiteMatterVol', 'CerebralWhiteMatterVol', 'SubCortGrayVol', '

### Regression Model (predict MRI structural measurements from exposure status)

How well can each exposure explain the variation in MRI values across individuals at one point in time.

10-fold cross-validation.


w/o covariates

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error

# ===== CONFIG =====
N_SPLITS = 10   # 10-fold CV
RANDOM_STATE = 42

# Custom scorers for RMSE and MAE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

results_cv = []

for mri_target in target:
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature].apply(pd.to_numeric, errors='coerce')

    # Drop missing values in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Find top exposure based on correlation with target
    corr_values = X_exposures.corrwith(y)
    top_exposure = corr_values.abs().idxmax()

    X_top = X_exposures[[top_exposure]]

    # Step 2: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 3: Cross-validation
    model = LinearRegression()
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    r2_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring="r2")
    rmse_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring=rmse_scorer)
    mae_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring=mae_scorer)

    results_cv.append({
        "MRI Target": mri_target,
        "Top Exposure": top_exposure,
        "R² Mean": np.mean(r2_scores),
        "RMSE Mean": -np.mean(rmse_scores),  # negate because greater_is_better=False
        "MAE Mean": -np.mean(mae_scores),    # same here
        "Mean MRI Value": y.mean()               # <- NEW: average MRI value
    })

# Results dataframe
results_cv_df = pd.DataFrame(results_cv).sort_values("R² Mean", ascending=False)

# Format
results_cv_df = results_cv_df.round({
    "R² Mean": 3,
    "RMSE Mean": 3,
    "MAE Mean": 3,
    "Mean MRI Value": 3
})

pd.set_option('display.max_rows', None)
results_cv_df.head()


,MRI Target,Top Exposure,R² Mean,RMSE Mean,MAE Mean,Mean MRI Value
174,ThickAvg_rh_medialorbitofrontal,expo_napp,0.072,0.146,0.115,2.510
26,Right-Hippocampus,expo_heat,0.021,423.194,352.149,4241.989
108,GrayVol_rh_parahippocampal,expo_vibr,0.013,293.394,232.786,2056.163
137,ThickAvg_lh_lateraloccipital,expo_bite,0.011,0.108,0.089,2.159
133,ThickAvg_lh_fusiform,expo_heat,0.010,0.120,0.098,2.742


W/ covariates

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error

# ===== CONFIG =====
N_SPLITS = 10   # 10-fold CV
RANDOM_STATE = 42

# Custom scorers for RMSE and MAE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Remove non-MRI targets (e.g., Age, Sex)
exclude_targets = ["Age", "Sex"]
mri_targets = [t for t in target if t not in exclude_targets]

results_cv = []

for mri_target in mri_targets:   # use filtered list here
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature + covariate_cols].apply(pd.to_numeric, errors='coerce')

    # Drop missing values in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Find top exposure based on correlation with target
    corr_values = X_exposures.corrwith(y)
    top_exposure = corr_values.abs().idxmax()

    X_top = X_exposures[[top_exposure]]

    # Step 2: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 3: Cross-validation
    model = LinearRegression()
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    r2_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring="r2")
    rmse_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring=rmse_scorer)
    mae_scores = cross_val_score(model, X_top_std, y, cv=kf, scoring=mae_scorer)

    results_cv.append({
        "MRI Target": mri_target,
        "Top Exposure": top_exposure,
        "R² Mean": np.mean(r2_scores),
        "RMSE Mean": -np.mean(rmse_scores),
        "MAE Mean": -np.mean(mae_scores),
        "Mean MRI Value": y.mean()
    })

# Results dataframe
results_cv_df = pd.DataFrame(results_cv).sort_values("R² Mean", ascending=False).round({
    "R² Mean": 3,
    "RMSE Mean": 3,
    "MAE Mean": 3,
    "Mean MRI Value": 3
})

pd.set_option('display.max_rows', None)
results_cv_df.head()


,MRI Target,Top Exposure,R² Mean,RMSE Mean,MAE Mean,Mean MRI Value
154,ThickAvg_lh_superiorfrontal,Age,0.196,0.126,0.105,2.741
144,ThickAvg_lh_parsopercularis,Age,0.182,0.120,0.098,2.597
188,ThickAvg_rh_superiorfrontal,Age,0.180,0.118,0.095,2.700
153,ThickAvg_lh_rostralmiddlefrontal,Age,0.171,0.123,0.101,2.469
195,ThickAvg_rh_insula,Age,0.170,0.149,0.120,3.012


W/ Covariates adjusted.

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error

# ===== CONFIG =====
N_SPLITS = 10   # 10-fold CV
RANDOM_STATE = 42

# Custom scorers for RMSE and MAE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

results_cv = []

for mri_target in target:
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature].apply(pd.to_numeric, errors='coerce')
    X_covariates = df[covariate_cols].apply(pd.to_numeric, errors='coerce')

    # Drop missing values in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]
    X_covariates = X_covariates.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Residualize MRI target by covariates
    cov_imputer = SimpleImputer(strategy='median')
    X_cov_imputed = cov_imputer.fit_transform(X_covariates)
    cov_scaler = StandardScaler()
    X_cov_std = cov_scaler.fit_transform(X_cov_imputed)

    cov_model = LinearRegression()
    cov_model.fit(X_cov_std, y)
    y_resid = y - cov_model.predict(X_cov_std)

    # Step 2: Find top exposure based on correlation with residuals
    corr_values = X_exposures.corrwith(pd.Series(y_resid, index=y.index))
    top_exposure = corr_values.abs().idxmax()

    X_top = X_exposures[[top_exposure]]

    # Step 3: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 4: Cross-validation using residualized y
    model = LinearRegression()
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    r2_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring="r2")
    rmse_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring=rmse_scorer)
    mae_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring=mae_scorer)

    results_cv.append({
        "MRI Target": mri_target,
        "Top Exposure": top_exposure,
        "R² Mean": np.mean(r2_scores),
        "RMSE Mean": -np.mean(rmse_scores),
        "MAE Mean": -np.mean(mae_scores),
        "Mean MRI Value": y.mean()   # swapped Min/Max for average MRI value
    })

# Results dataframe
results_cv_df = pd.DataFrame(results_cv).sort_values("MAE Mean", ascending=True)

# Format
results_cv_df = results_cv_df.round({
    "R² Mean": 3,
    "RMSE Mean": 3,
    "MAE Mean": 3,
    "Mean MRI Value": 3
})

pd.set_option('display.max_rows', None)
results_cv_df.head()


,MRI Target,Top Exposure,R² Mean,RMSE Mean,MAE Mean,Mean MRI Value
196,Sex,expo_vibr,-0.046,0.000,0.000,0.867
197,Age,expo_vibr,-0.047,0.000,0.000,57.050
54,BrainSegVol-to-eTIV,expo_nois,-0.027,0.028,0.022,0.779
55,MaskVol-to-eTIV,expo_napp,-0.072,0.032,0.025,1.054
155,ThickAvg_lh_superiorparietal,expo_nois,-0.024,0.095,0.075,2.158


#### Lasso, Ridge and Elastic Net Regression (w covariates).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# ===== CONFIG =====
N_SPLITS = 10   # 10-fold CV
RANDOM_STATE = 42

results_cv = []

for mri_target in target:
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature].apply(pd.to_numeric, errors='coerce')
    X_covariates = df[covariate_cols].apply(pd.to_numeric, errors='coerce')

    # Drop missing values in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]
    X_covariates = X_covariates.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Residualize MRI target by covariates
    cov_imputer = SimpleImputer(strategy='median')
    X_cov_imputed = cov_imputer.fit_transform(X_covariates)
    cov_scaler = StandardScaler()
    X_cov_std = cov_scaler.fit_transform(X_cov_imputed)

    cov_model = LinearRegression()
    cov_model.fit(X_cov_std, y)
    y_resid = y - cov_model.predict(X_cov_std)

    # Step 2: Find top exposure based on correlation with residuals
    corr_values = X_exposures.corrwith(pd.Series(y_resid, index=y.index))
    top_exposure = corr_values.abs().idxmax()
    X_top = X_exposures[[top_exposure]]

    # Step 3: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 4: Cross-validation for multiple models
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    models = {
        "Linear": LinearRegression(),
        "Ridge": Ridge(alpha=1.0, random_state=RANDOM_STATE),
        "Lasso": Lasso(alpha=0.01, random_state=RANDOM_STATE, max_iter=10000),
        "ElasticNet": ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=RANDOM_STATE, max_iter=10000)
    }

    scores = {}
    for name, model in models.items():
        cv_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring="r2")
        scores[name] = np.mean(cv_scores)

    results_cv.append({
        "MRI Target": mri_target,
        "Top Exposure": top_exposure,
        "R² Linear": scores["Linear"],
        "R² Ridge": scores["Ridge"],
        "R² Lasso": scores["Lasso"],
        "R² ElasticNet": scores["ElasticNet"]
    })

# Results dataframe
results_cv_df = pd.DataFrame(results_cv).sort_values("R² Linear", ascending=False)

# Format
results_cv_df = results_cv_df.round({
    "R² Linear": 3,
    "R² Ridge": 3,
    "R² Lasso": 3,
    "R² ElasticNet": 3
})

pd.set_option('display.max_rows', None)
results_cv_df.head()


,MRI Target,Top Exposure,R² Linear,R² Ridge,R² Lasso,R² ElasticNet
174,ThickAvg_rh_medialorbitofrontal,expo_napp,0.017,0.017,0.010,0.015
137,ThickAvg_lh_lateraloccipital,expo_bite,0.016,0.016,0.002,0.012
171,ThickAvg_rh_lateraloccipital,expo_bite,0.001,0.001,-0.007,-0.000
160,ThickAvg_lh_transversetemporal,expo_bite,0.000,0.000,-0.006,-0.002
194,ThickAvg_rh_transversetemporal,expo_sand,-0.001,-0.001,-0.004,-0.002


#### Non-linear models.


##### Random Forest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

# ===== CONFIG =====
N_SPLITS = 10
RANDOM_STATE = 42

# Custom scorers
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

results_rf = []

for mri_target in target:
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature].apply(pd.to_numeric, errors='coerce')
    X_covariates = df[covariate_cols].apply(pd.to_numeric, errors='coerce')

    # Drop missing in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]
    X_covariates = X_covariates.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Residualize MRI by covariates (linear adjustment)
    cov_imputer = SimpleImputer(strategy='median')
    X_cov_imputed = cov_imputer.fit_transform(X_covariates)
    cov_scaler = StandardScaler()
    X_cov_std = cov_scaler.fit_transform(X_cov_imputed)

    cov_model = LinearRegression()
    cov_model.fit(X_cov_std, y)
    y_resid = y - cov_model.predict(X_cov_std)

    # Step 2: Pick top exposure (highest correlation with residuals)
    corr_values = X_exposures.corrwith(pd.Series(y_resid, index=y.index))
    top_exposure = corr_values.abs().idxmax()
    X_top = X_exposures[[top_exposure]]

    # Step 3: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 4: Random Forest with 10-fold CV
    model = RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        random_state=RANDOM_STATE
    )
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    r2_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring="r2")
    rmse_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring=rmse_scorer)
    mae_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring=mae_scorer)

    results_rf.append({
        "MRI Target": mri_target,
        "Top Exposure": top_exposure,
        "R² Mean": np.mean(r2_scores),
        "RMSE Mean": -np.mean(rmse_scores),
        "MAE Mean": -np.mean(mae_scores),
        "Mean MRI Value": y.mean()
    })

# Results dataframe
results_rf_df = pd.DataFrame(results_rf).sort_values("R² Mean", ascending=False)
results_rf_df = results_rf_df.round({
    "R² Mean": 3,
    "RMSE Mean": 3,
    "MAE Mean": 3,
    "Mean MRI Value": 3
})

pd.set_option('display.max_rows', None)
results_rf_df.head()


,MRI Target,Top Exposure,R² Mean,RMSE Mean,MAE Mean,Mean MRI Value
137,ThickAvg_lh_lateraloccipital,expo_bite,0.016,0.101,0.081,2.159
174,ThickAvg_rh_medialorbitofrontal,expo_napp,0.016,0.136,0.109,2.510
171,ThickAvg_rh_lateraloccipital,expo_bite,0.001,0.094,0.079,2.267
160,ThickAvg_lh_transversetemporal,expo_bite,-0.000,0.187,0.152,2.420
194,ThickAvg_rh_transversetemporal,expo_sand,-0.001,0.200,0.157,2.468


Other non-linear models.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import LinearRegression

# ===== CONFIG =====
N_SPLITS = 10
RANDOM_STATE = 42

# Custom RMSE scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

results = []

# Loop over MRI targets
for mri_target in target:
    y = pd.to_numeric(df[mri_target], errors='coerce')
    X_exposures = df[feature].apply(pd.to_numeric, errors='coerce')
    X_covariates = df[covariate_cols].apply(pd.to_numeric, errors='coerce')

    # Drop missing in target
    mask = y.notna()
    y = y.loc[mask]
    X_exposures = X_exposures.loc[mask]
    X_covariates = X_covariates.loc[mask]

    if y.shape[0] == 0:
        continue

    # Step 1: Residualize MRI by covariates (linear adjustment)
    cov_imputer = SimpleImputer(strategy='median')
    X_cov_imputed = cov_imputer.fit_transform(X_covariates)
    cov_scaler = StandardScaler()
    X_cov_std = cov_scaler.fit_transform(X_cov_imputed)

    cov_model = LinearRegression()
    cov_model.fit(X_cov_std, y)
    y_resid = y - cov_model.predict(X_cov_std)

    # Step 2: Pick top exposure (highest correlation with residuals)
    corr_values = X_exposures.corrwith(pd.Series(y_resid, index=y.index))
    top_exposure = corr_values.abs().idxmax()
    X_top = X_exposures[[top_exposure]]

    # Step 3: Impute + scale exposures
    exp_imputer = SimpleImputer(strategy='median')
    X_top_imputed = exp_imputer.fit_transform(X_top)
    exp_scaler = StandardScaler()
    X_top_std = exp_scaler.fit_transform(X_top_imputed)

    # Step 4: Define models
    models = {
        "R² Random Forest": RandomForestRegressor(n_estimators=500, random_state=RANDOM_STATE),
        "R² Gradient Boosting": GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=RANDOM_STATE
        ),
        "R² SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
        "R² KNN": KNeighborsRegressor(n_neighbors=5, weights="distance"),
    }

    # Step 5: Cross-validation for each model
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    row = {"MRI Target": mri_target, "Top Exposure": top_exposure}
    for model_name, model in models.items():
        r2_scores = cross_val_score(model, X_top_std, y_resid, cv=kf, scoring="r2")
        row[model_name] = np.mean(r2_scores)
    results.append(row)

# Results dataframe (wide format, only R²)
results_df = pd.DataFrame(results).round(3)
results_df = results_df.sort_values(by=["R² Random Forest"], ascending=False)

pd.set_option('display.max_rows', None)
results_df.head()


,MRI Target,Top Exposure,R² Random Forest,R² Gradient Boosting,R² SVR,R² KNN
137,ThickAvg_lh_lateraloccipital,expo_bite,0.016,0.016,0.019,-0.152
174,ThickAvg_rh_medialorbitofrontal,expo_napp,0.016,0.017,0.019,-0.405
171,ThickAvg_rh_lateraloccipital,expo_bite,0.001,0.001,-0.003,-0.114
160,ThickAvg_lh_transversetemporal,expo_bite,-0.000,0.000,0.005,-0.287
194,ThickAvg_rh_transversetemporal,expo_sand,-0.001,-0.001,-0.003,-0.585
